In [35]:
import pandas as pd
import tqdm
import glob
import os

In [36]:
files = glob.glob("../data/wikipedia/a.parquet")
files = [f for f in files if "all" not in f]

In [43]:
def output_seperated_files(token_length, stride, drop_category):    
    output_dir = f"../data/wikipedia/sep4/token_length{token_length}_stride{stride}_drop_cat{drop_category}"
    os.makedirs(output_dir, exist_ok=True)
    for file in files:
        df_wiki = pd.read_parquet(file)
        df_wiki_sep = []
        for i in tqdm.tqdm(range(len(df_wiki))):
            series = df_wiki.iloc[i]
            text = series["text"]
            if drop_category:
                idx = text.find(" Category:")
                text = text[:idx]
            
            text_space = text.split(" ")

            # タイトルだけで検索もひっかかるようにしとく
            series_ = series.to_dict()
            series_["text"] = " ".join(text_space[:token_length])
            series_["title_only"] = True
            df_wiki_sep.append(series_)
            
            for j in range(0, len(text_space), stride):
                series_ = series.to_dict()
                series_["text"] = " ".join(text_space[j:j+token_length])
                series_["title_only"] = False
                if series_["text"] != "":
                    df_wiki_sep.append(series_)
        df_wiki_sep = pd.DataFrame(df_wiki_sep)
        df_wiki_sep.to_parquet(f"{output_dir}/{os.path.basename(file)}")

In [44]:
output_seperated_files(token_length=100, stride=75, drop_category=True)

100%|████████████████████████████████████████████████████████████████████████| 442726/442726 [01:16<00:00, 5792.09it/s]


In [46]:
output_seperated_files(token_length=100, stride=75, drop_category=False)

100%|████████████████████████████████████████████████████████████████████████| 442726/442726 [01:12<00:00, 6087.48it/s]


In [54]:
output_seperated_files(token_length=150, stride=100, drop_category=True)

100%|████████████████████████████████████████████████████████████████████████| 442726/442726 [01:11<00:00, 6199.67it/s]
